In [10]:

from langsmith import traceable
from groq import Groq
from typing import List, Optional
import json

groq_client = Groq(api_key="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")
GROQ_MODEL = "deepseek-r1-distill-llama-70b"

@traceable(
  run_type="tool"
)
def get_current_temperature(location: str, unit: str):
    return 65 if unit == "Fahrenheit" else 17

@traceable(run_type="llm")
def call_groq(
    messages: List[dict], tools: Optional[List[dict]] = None
) -> str:
  if tools:
      return groq_client.chat.completions.create(
        model=GROQ_MODEL,
        messages=messages,
        temperature=0,
        tools=tools
      )
  else:
      return groq_client.chat.completions.create(
        model=GROQ_MODEL,
        messages=messages,
        temperature=0,
      )

@traceable(run_type="chain")
def ask_about_the_weather(inputs, tools):
  response = call_groq(inputs, tools)
  
  if response.choices and response.choices[0].message.tool_calls:
      tool_call = response.choices[0].message.tool_calls[0]
      tool_call_args = json.loads(tool_call.function.arguments)

In [9]:

from langsmith import traceable

def _convert_docs(results):
  return [
      {
          "page_content": r,
          "type": "Document",
          "metadata": {"foo": "bar"}
      }
      for r in results
  ]

@traceable(
    run_type="retriever"
)
def retrieve_docs(query):
  contents = ["Document contents 1", "Document contents 2", "Document contents 3"]
  return _convert_docs(contents)

retrieve_docs("User query")


[{'page_content': 'Document contents 1',
  'type': 'Document',
  'metadata': {'foo': 'bar'}},
 {'page_content': 'Document contents 2',
  'type': 'Document',
  'metadata': {'foo': 'bar'}},
 {'page_content': 'Document contents 3',
  'type': 'Document',
  'metadata': {'foo': 'bar'}}]

In [8]:

def _reduce_chunks(chunks: list):
    all_text = "".join([chunk["choices"][0]["message"]["content"] for chunk in chunks])
    return {"choices": [{"message": {"content": all_text, "role": "assistant"}}]}

@traceable(
    run_type="llm",
    metadata={"ls_provider": "my_provider", "ls_model_name": "my_model"},
    reduce_fn=_reduce_chunks,
)
def my_streaming_chat_model(messages: list):
    for chunk in ["Hello, " + messages[1]["content"]]:
        yield {
            "choices": [
                {
                    "message": {
                        "content": chunk,
                        "role": "assistant",
                    }
                }
            ]
        }

list(
    my_streaming_chat_model(
        [
            {"role": "system", "content": "You are a helpful assistant. Please greet the user."},
            {"role": "user", "content": "polly the parrot"},
        ],
    )
)

[{'choices': [{'message': {'content': 'Hello, polly the parrot',
     'role': 'assistant'}}]}]

In [7]:

from langsmith import traceable

inputs = [
  {"role": "system", "content": "You are a helpful assistant."},
  {"role": "user", "content": "I'd like to book a table for two."},
]

output = {
  "choices": [
      {
          "message": {
              "role": "assistant",
              "content": "Sure, what time would you like to book the table for?"
          }
      }
  ]
}

@traceable(
  run_type="llm", metadata={"ls_provider": "my_provider", "ls_model_name": "my_model"}
)
def chat_model(messages: list):
  return output

chat_model(inputs)


{'choices': [{'message': {'role': 'assistant',
    'content': 'Sure, what time would you like to book the table for?'}}]}

In [6]:

from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

False